BOS behavior
===

Mimicks what already exists on BOS

In [52]:
import ast
import jinja2
import jinja2schema
import jmespath
import json
import pprint
import urllib.request

def keys(r):
    return jinja2schema.infer(r).keys()

default_api = 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/route/status/{{dep_airport_iata}}/{{arr_airport_iata}}/dep/{{ dep_date }}?appId=a7f68539&appKey=d4350ab17849a1142a647223dd9d7a13&hourOfDay=&utc=true&numHours=24&maxFlights=99'
defaults = {
    'dep_airport_iata': 'ORY',
    'arr_airport_iata': 'RUN',
    'dep_date': '2018/08/30'
}
default_extraction = '{"flights": "flightStatuses[]", "appendix": "appendix"}'

api = input('API: ') or default_api

vars_ = {}
for var in keys(api):
    val = input(var + ': ') or defaults[var]
    try:
        vars_[var] = ast.literal_eval(val)
    except ValueError:
        vars_[var] = val
    except SyntaxError:
        vars_[var] = val

template = jinja2.Template(api)
api_url = template.render(**vars_)
print(api_url)

api_http = urllib.request.urlopen(api_url)
api_res = api_http.read().decode('utf-8')
json = json.loads(api_res)

extractions = ast.literal_eval(input('Extractions (dict):') or default_extraction)
outputs = {}
for name, exp in extractions.items():
    expression = jmespath.compile(exp)
    outputs[name] = expression.search(json)
print('\nExtractions')
pprint.pprint(outputs)

API: 
dep_airport_iata: 
arr_airport_iata: 
dep_date: 
https://api.flightstats.com/flex/flightstatus/rest/v2/json/route/status/ORY/RUN/dep/2018/08/30?appId=a7f68539&appKey=d4350ab17849a1142a647223dd9d7a13&hourOfDay=&utc=true&numHours=24&maxFlights=99
Extractions (dict):

Extractions
{'appendix': {'airlines': [{'active': True,
                            'fs': 'SS',
                            'iata': 'SS',
                            'icao': 'CRL',
                            'name': 'Corsair'},
                           {'active': True,
                            'fs': 'TX',
                            'iata': 'TX',
                            'icao': 'FWI',
                            'name': 'Air Caraibes',
                            'phoneNumber': '877.772-1005'},
                           {'active': True,
                            'fs': 'AF',
                            'iata': 'AF',
                            'icao': 'AFR',
                            'name': 'Air France',

Merging
===

- [ ] User should be able to choose between an API or a JSON in the 'API' input.
- [ ] Manage loops.
- [ ] Build Graphene schema.

In [78]:
import ast
from dotted.collection import DottedCollection
import jinja2
import jinja2schema
import jmespath
import json
import pprint
import urllib.request


# Default values for tests
default_api = 'https://api.flightstats.com/flex/flightstatus/rest/v2/json/route/status/{{dep_airport_iata}}/{{arr_airport_iata}}/dep/{{ dep_date }}?appId=a7f68539&appKey=d4350ab17849a1142a647223dd9d7a13&hourOfDay=&utc=true&numHours=24&maxFlights=99'
defaults = {
    'dep_airport_iata': 'ORY',
    'arr_airport_iata': 'RUN',
    'dep_date': '2018/08/30'
}
default_path = 'request.departureAirport.requestedCode'

default_api2 = default_api
defaults2 = {
    'dep_airport_iata': 'CDG',
    'arr_airport_iata': 'RUN',
    'dep_date': '2018/08/30'
}
default_path2 = default_path


def keys(r):
    return jinja2schema.infer(r).keys()

def api_to_json(default_api=None, defaults=None):
    api = input('API: ') or default_api

    vars_ = {}
    for var in keys(api):
        val = input(var + ': ') or defaults[var]
        try:
            vars_[var] = ast.literal_eval(val)
        except ValueError:
            vars_[var] = val
        except SyntaxError:
            vars_[var] = val

    template = jinja2.Template(api)
    api_url = template.render(**vars_)
    print(api_url)

    api_http = urllib.request.urlopen(api_url)
    api_res = api_http.read().decode('utf-8')
    return json.loads(api_res)

json_dict = api_to_json(default_api, defaults)
json_dict2 = api_to_json(default_api2, defaults2)


## Merging

json_dict = DottedCollection.factory(json_dict)
json_dict2 = DottedCollection.factory(json_dict2)

print('Merging (paths with following syntax: dict.item.list_index.subitem)')
path = input('Path for second JSON in first one: ') or default_path or '@'
path2 = input('Path in second JSON: ') or default_path2 or '@'

if isinstance(json_dict[path], dict):
    json_dict[path].update(json_dict2[path2])
else:
    json_dict[path] = json_dict2[path2]

json_dict = json_dict.to_python()

## Build schema
# build_graphene_schema(json)


API: 
dep_airport_iata: 
dep_date: 
arr_airport_iata: 
https://api.flightstats.com/flex/flightstatus/rest/v2/json/route/status/ORY/RUN/dep/2018/08/30?appId=a7f68539&appKey=d4350ab17849a1142a647223dd9d7a13&hourOfDay=&utc=true&numHours=24&maxFlights=99
API: 
dep_airport_iata: 
dep_date: 
arr_airport_iata: 
https://api.flightstats.com/flex/flightstatus/rest/v2/json/route/status/CDG/RUN/dep/2018/08/30?appId=a7f68539&appKey=d4350ab17849a1142a647223dd9d7a13&hourOfDay=&utc=true&numHours=24&maxFlights=99
Merging (paths with followingsyntax syntax: dict.item.list_index.subitem)
Path for second JSON in first one: 
Path in second JSON: 


KeyError: 'request.departureAirport.requestedCode'

Test
===

Without inputs or API

In [84]:
import ast
import copy
from dotted.collection import DottedCollection
import jinja2
import jinja2schema
import jmespath
import json
import pprint
import urllib.request

def keys(r):
    return jinja2schema.infer(r).keys()

data = {"request":{"departureAirport":{"requestedCode":"ORY","fsCode":"ORY"},"arrivalAirport":{"requestedCode":"RUN","fsCode":"RUN"},"date":{"year":"2018","month":"8","day":"30","interpreted":"2018-08-30"},"hourOfDay":{"requested":"","interpreted":0},"utc":{"requested":"True","interpreted":True},"numHours":{"requested":"24","interpreted":24},"codeType":{},"maxFlights":{"requested":"99","interpreted":99},"extendedOptions":{},"url":"https://api.flightstats.com/flex/flightstatus/rest/v2/json/route/status/ORY/RUN/dep/2018/08/30"},"appendix":{"airlines":[{"fs":"SS","iata":"SS","icao":"CRL","name":"Corsair","active":True},{"fs":"TX","iata":"TX","icao":"FWI","name":"Air Caraibes","phoneNumber":"877.772-1005","active":True},{"fs":"AF","iata":"AF","icao":"AFR","name":"Air France","phoneNumber":"1-800-237-2747","active":True},{"fs":"B2F","iata":"BF","icao":"FBU","name":"French Bee","active":True}],"airports":[{"fs":"RUN","iata":"RUN","icao":"FMEE","name":"Reunion Roland Garros Airport","city":"Saint Denis de la Reunion","cityCode":"RUN","countryCode":"RE","countryName":"Reunion","regionName":"Africa","timeZoneRegionName":"Indian/Reunion","localTime":"2018-08-29T20:43:46.598","utcOffsetHours":4.0,"latitude":-20.892,"longitude":55.511877,"elevationFeet":66,"classification":4,"active":True,"delayIndexUrl":"https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/RUN?codeType=fs","weatherUrl":"https://api.flightstats.com/flex/weather/rest/v1/json/all/RUN?codeType=fs"},{"fs":"MRU","iata":"MRU","icao":"FIMP","name":"Sir Seewoosagur Ramgoolam International Airport","city":"Mauritius","cityCode":"MRU","countryCode":"MU","countryName":"Mauritius","regionName":"Africa","timeZoneRegionName":"Indian/Mauritius","localTime":"2018-08-29T20:43:46.611","utcOffsetHours":4.0,"latitude":-20.431998,"longitude":57.676629,"elevationFeet":212,"classification":3,"active":True,"delayIndexUrl":"https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/MRU?codeType=fs","weatherUrl":"https://api.flightstats.com/flex/weather/rest/v1/json/all/MRU?codeType=fs"},{"fs":"ORY","iata":"ORY","icao":"LFPO","name":"Paris Orly Airport","street1":"94396 Orly Aérogare","city":"Paris","cityCode":"PAR","countryCode":"FR","countryName":"France","regionName":"Europe","timeZoneRegionName":"Europe/Paris","localTime":"2018-08-29T18:43:46.598","utcOffsetHours":2.0,"latitude":48.728283,"longitude":2.3597,"elevationFeet":292,"classification":1,"active":True,"delayIndexUrl":"https://api.flightstats.com/flex/delayindex/rest/v1/json/airports/ORY?codeType=fs","weatherUrl":"https://api.flightstats.com/flex/weather/rest/v1/json/all/ORY?codeType=fs"}],"equipments":[{"iata":"332","name":"Airbus A330-200","turboProp":False,"jet":True,"widebody":True,"regional":False},{"iata":"359","name":"Airbus A350-900","turboProp":False,"jet":True,"widebody":True,"regional":False},{"iata":"77W","name":"Boeing 777-300ER","turboProp":False,"jet":True,"widebody":True,"regional":False}]},"flightStatuses":[{"flightId":971531287,"carrierFsCode":"B2F","flightNumber":"702","departureAirportFsCode":"ORY","arrivalAirportFsCode":"RUN","departureDate":{"dateLocal":"2018-08-30T16:35:00.000","dateUtc":"2018-08-30T14:35:00.000Z"},"arrivalDate":{"dateLocal":"2018-08-31T05:30:00.000","dateUtc":"2018-08-31T01:30:00.000Z"},"status":"S","schedule":{"flightType":"J","serviceClasses":"RFJY","restrictions":""},"operationalTimes":{"publishedDeparture":{"dateLocal":"2018-08-30T16:35:00.000","dateUtc":"2018-08-30T14:35:00.000Z"},"publishedArrival":{"dateLocal":"2018-08-31T05:30:00.000","dateUtc":"2018-08-31T01:30:00.000Z"},"scheduledGateDeparture":{"dateLocal":"2018-08-30T16:35:00.000","dateUtc":"2018-08-30T14:35:00.000Z"},"scheduledGateArrival":{"dateLocal":"2018-08-31T05:30:00.000","dateUtc":"2018-08-31T01:30:00.000Z"}},"codeshares":[{"fsCode":"TX","flightNumber":"6702","relationship":"L"}],"flightDurations":{"scheduledBlockMinutes":655},"airportResources":{"departureTerminal":"S"},"flightEquipment":{"scheduledEquipmentIataCode":"359"}},{"flightId":971521974,"carrierFsCode":"AF","flightNumber":"644","departureAirportFsCode":"ORY","arrivalAirportFsCode":"RUN","departureDate":{"dateLocal":"2018-08-30T17:55:00.000","dateUtc":"2018-08-30T15:55:00.000Z"},"arrivalDate":{"dateLocal":"2018-08-31T06:50:00.000","dateUtc":"2018-08-31T02:50:00.000Z"},"status":"S","schedule":{"flightType":"J","serviceClasses":"RFJY","restrictions":""},"operationalTimes":{"publishedDeparture":{"dateLocal":"2018-08-30T17:55:00.000","dateUtc":"2018-08-30T15:55:00.000Z"},"publishedArrival":{"dateLocal":"2018-08-31T06:50:00.000","dateUtc":"2018-08-31T02:50:00.000Z"},"scheduledGateDeparture":{"dateLocal":"2018-08-30T17:55:00.000","dateUtc":"2018-08-30T15:55:00.000Z"},"scheduledGateArrival":{"dateLocal":"2018-08-31T06:50:00.000","dateUtc":"2018-08-31T02:50:00.000Z"}},"flightDurations":{"scheduledBlockMinutes":655},"airportResources":{"departureTerminal":"W"},"flightEquipment":{"scheduledEquipmentIataCode":"77W","actualEquipmentIataCode":"77W"}},{"flightId":971521990,"carrierFsCode":"AF","flightNumber":"642","departureAirportFsCode":"ORY","arrivalAirportFsCode":"RUN","departureDate":{"dateLocal":"2018-08-30T21:00:00.000","dateUtc":"2018-08-30T19:00:00.000Z"},"arrivalDate":{"dateLocal":"2018-08-31T09:55:00.000","dateUtc":"2018-08-31T05:55:00.000Z"},"status":"S","schedule":{"flightType":"J","serviceClasses":"RFJY","restrictions":""},"operationalTimes":{"publishedDeparture":{"dateLocal":"2018-08-30T21:00:00.000","dateUtc":"2018-08-30T19:00:00.000Z"},"publishedArrival":{"dateLocal":"2018-08-31T09:55:00.000","dateUtc":"2018-08-31T05:55:00.000Z"},"scheduledGateDeparture":{"dateLocal":"2018-08-30T21:00:00.000","dateUtc":"2018-08-30T19:00:00.000Z"},"scheduledGateArrival":{"dateLocal":"2018-08-31T09:55:00.000","dateUtc":"2018-08-31T05:55:00.000Z"}},"flightDurations":{"scheduledBlockMinutes":655},"airportResources":{"departureTerminal":"W"},"flightEquipment":{"scheduledEquipmentIataCode":"77W"}},{"flightId":971590262,"carrierFsCode":"SS","flightNumber":"710","departureAirportFsCode":"ORY","arrivalAirportFsCode":"RUN","departureDate":{"dateLocal":"2018-08-30T21:20:00.000","dateUtc":"2018-08-30T19:20:00.000Z"},"arrivalDate":{"dateLocal":"2018-08-31T10:45:00.000","dateUtc":"2018-08-31T06:45:00.000Z"},"status":"S","schedule":{"flightType":"J","serviceClasses":"RFJY","restrictions":"","downlines":[{"fsCode":"MRU","flightId":971590304}]},"operationalTimes":{"publishedDeparture":{"dateLocal":"2018-08-30T21:20:00.000","dateUtc":"2018-08-30T19:20:00.000Z"},"publishedArrival":{"dateLocal":"2018-08-31T10:45:00.000","dateUtc":"2018-08-31T06:45:00.000Z"},"scheduledGateDeparture":{"dateLocal":"2018-08-30T21:20:00.000","dateUtc":"2018-08-30T19:20:00.000Z"},"scheduledGateArrival":{"dateLocal":"2018-08-31T10:45:00.000","dateUtc":"2018-08-31T06:45:00.000Z"}},"flightDurations":{"scheduledBlockMinutes":685},"airportResources":{"departureTerminal":"S"},"flightEquipment":{"scheduledEquipmentIataCode":"332"}}]}
data2 = data.copy()

default_path = 'request.departureAirport.requestedCode'
default_path2 = default_path

json_dict = DottedCollection.factory(data)
json_dict2 = DottedCollection.factory(data2)

json_dict2[path2] = 'CDG'

if isinstance(json_dict[path], dict):
    json_dict[path].update(json_dict2[path2])
else:
    json_dict[path] = json_dict2[path2]

json_dict = json_dict.to_python()

JSON Cleaning
===

In [77]:
"""Make JSON acceptable by GraphQL.

Replace special characters in JSON keys to be accepted by GraphQL.
The dictionary `key_map` allows mapping between old and new keys.
"""
import re
import unidecode


class JSonCleaning:

    key_map = {}
    reversed_key_map = {}

    @classmethod
    def strip_special_char(cls, txt):
        """Convert a string by removing diacritics and replacing special characters with underscores.

        Arguments:
            txt {string} -- text to be transformed

        Returns:
            string -- new text that complies with GraphQL key convention: `Names must match /^[_a-zA-Z][_a-zA-Z0-9]*$/`
        """
        if txt in cls.key_map:
            return cls.key_map[txt]
        unaccented = unidecode.unidecode(txt)
        unaccented = re.sub(r'\W','_',unaccented)
        if unaccented[0] in range(10):
            unaccented = '_' + unaccented
        new_txt = unaccented
        suffix = 0
        while new_txt in cls.key_map.values():
            new_txt = unaccented + str(suffix)
            suffix += 1
        cls.key_map[txt] = new_txt
        return new_txt

    @classmethod
    def restore_special_char(cls, txt):
        return cls.reversed_key_map[txt]


    @classmethod
    def _encode_decode_json_str(cls, json, func):
        pattern = re.compile(r'(?:(?:")([^\"]+)(?:":))|(?:(?:\')([^\']+)(?:\':))')
        match_iter = pattern.finditer(json)
        new_json = []
        last_end = 0
        for match in match_iter:
            n = 1 if match.group(1) else 2
            repl = func(match.group(n))
            beg, end = match.span(n)
            new_json.append(json[last_end:beg])
            new_json.append(repl)
            last_end = end
        new_json.append(json[last_end:])
        new_json = ''.join(new_json)
        return new_json

    @classmethod
    def _encode_decode_json_dict(cls, d, convert_function):
        """
        Convert a nested dictionary from one convention to another.

        Arguments:
            d {dict} -- dictionary (nested or not) to be converted.
            convert_function {func} -- function that takes the string in one convention and returns it in the other one.
        Returns:
            dict -- dictionary with the new keys.
        """
        new = {}
        for k, v in d.items():
            new_v = v
            if isinstance(v, dict):
                new_v = cls._encode_decode_json_dict(v, convert_function)
            elif isinstance(v, list):
                new_v = list()
                for x in v:
                    if isinstance(x, dict):
                        new_v.append(cls._encode_decode_json_dict(x, convert_function))
                    else:
                        new_v.append(x)
            new[convert_function(k)] = new_v
        return new

    @classmethod
    def clean_keys(cls, json):
        """Transform special characters from keys."""
        if isinstance(json, dict):
            return cls._encode_decode_json_dict(json, cls.strip_special_char)
        print(type(json))
        return cls._encode_decode_json_str(json, cls.strip_special_char)

    @classmethod
    def restore_keys(cls, json):
        """Convert keys from stripped to original ones."""
        cls.reversed_key_map = dict(map(reversed, cls.key_map.items()))
        if isinstance(json, dict):
            return cls._encode_decode_json_dict(json, cls.restore_special_char)
        return cls._encode_decode_json_str(json, cls.restore_special_char)


Schema builder
===

In [87]:
import genson
import graphene
from graphene.types.objecttype import ObjectTypeOptions, BaseOptions
import json
import requests
import sys

class SchemaBuilder:

    TYPE_TRANSLATION = {
        "object": graphene.ObjectType,
        "string": graphene.String,
        "array": graphene.List,
        "number": graphene.Float,
        "integer": graphene.Int,
        "boolean": graphene.Boolean
    }
    SCALARS = ["string", "number", "integer", "boolean"]

    def __init__(self):
        self.object_types = {}

    @staticmethod
    def json_to_json_schema(json_dict):
        builder = genson.SchemaBuilder()
        builder.add_schema({"type": "object", "properties": {}})
        builder.add_object(JSonCleaning.clean_keys(json_dict))
        schema = builder.to_json(indent=4)
        return json.loads(schema)

    def json_schema_to_graphene_object_type(self, name, schema):
        """:return : type, object"""
        if schema['type'] in SchemaBuilder.SCALARS:
            return 'scalar', SchemaBuilder.TYPE_TRANSLATION[schema['type']]
        elif schema['type'] == 'array':
            if 'items' in schema:
                _, T = self.json_schema_to_graphene_object_type(name, schema['items'])
                return 'array', T
            else:
                return 'array', graphene.String
        elif schema['type'] == 'object':
            if name in self.object_types:
                return 'object', self.object_types[name]
            options = ObjectTypeOptions(BaseOptions)
            options.fields = {'raw': graphene.Field(graphene.JSONString)}
            for field in schema['properties']:
                t, T = self.json_schema_to_graphene_object_type(field, schema['properties'][field])
                if t == 'array':
                    resolver = lambda self, info, field=field, T=T: [T(raw=sub_raw) for sub_raw in self.raw[field]]
                    options.fields[field] = graphene.Field(graphene.List(T), resolver=resolver)
                elif t == 'object':
                    resolver = lambda self, info, field=field, T=T: T(raw=self.raw[field])
                    options.fields[field] = graphene.Field(T, resolver=resolver)
                else:
                    resolver = lambda self, info, field=field: self.raw[field]
                    options.fields[field] = graphene.Field(T, resolver=resolver)
            object_type = graphene.ObjectType.create_type(class_name=name, _meta=options)
            self.object_types[name] = object_type
            return 'object', object_type
        else:
            print("UNKNOWN TYPE", schema['type'])

    def json_to_graphene_object_type(self, name, json_dict):
        _, T = self.json_schema_to_graphene_object_type(name, self.json_to_json_schema(json_dict))
        return T


# @mock.patch('requests.get', mock_get)
def build_query(json_dict):
    T = SchemaBuilder().json_to_graphene_object_type('test', json_dict)
    return T

# Global variables
T = build_query(json_dict)
raw = JSonCleaning.clean_keys(json_dict)


class Query(graphene.ObjectType):
    global T, raw
    a = graphene.Field(T)
    b = graphene.Field(graphene.Int)


    def resolve_a(self, info):
        return T(raw=raw)

    def resolve_b(self, info):
        return 1


schema = graphene.Schema(query=Query, auto_camelcase=False)


KeyError: 'properties'